# Magenta魔改记-1：MIDI文件读取

Magenta支持MIDI（.mid/.midi）文件与MusicXML（.xml/.mxl）、ABC数据（http://abcnotation.com，没有测试过）文件做训练数据。
通常，制作数据集的步骤是，先将原始文件转化成单个tfrecord文件保存，再根据每个不同的模型进行不同的数据清洗与处理。

那么这篇文章先着重分析第一步，将MIDI/MusicXML文件直接转换成tfrecord：
对应github中的提示：
https://github.com/tensorflow/magenta/tree/master/magenta/scripts#building-your-dataset

上述链接中的命令行如下：

```
INPUT_DIRECTORY=<folder containing MIDI and/or MusicXML files. can have child folders.>

# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD=/tmp/notesequences.tfrecord

convert_dir_to_note_sequences \
  --input_dir=$INPUT_DIRECTORY \
  --output_file=$SEQUENCES_TFRECORD \
  --recursive
```

这一步的bazel命令行如下（摘自源代码注释）：

```
Example usage:
  $ bazel build magenta/scripts:convert_dir_to_note_sequences

  $ ./bazel-bin/magenta/scripts/convert_dir_to_note_sequences \
    --input_dir=/path/to/input/dir \
    --output_file=/path/to/tfrecord/file \
    --num_threads=4 \
--log=INFO
```

可以看到，两个命令行的参数内容都不同，可见Magenta项目组对于文档或API的介绍并没有进行认真的维护。
 
## 魔改-1.0：


那么下面介绍如何修改这一步预处理的参数。

这一步运行的文件位置如下：
https://github.com/tensorflow/magenta/blob/master/magenta/scripts/convert_dir_to_note_sequences.py



打开源代码我们可以看到，程序一开始就定义了一系列tf.flag：

```
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('input_dir', None,
                           'Directory containing files to convert.')
#输入路径
tf.app.flags.DEFINE_string('output_file', None,
                           'Path to output TFRecord file. Will be overwritten '
                           'if it already exists.')
#输出路径
tf.app.flags.DEFINE_bool('recursive', False,
                         'Whether or not to recurse into subdirectories.')
#是否递归查找子路径的文件
tf.app.flags.DEFINE_integer('num_threads', 1,
                            'Number of worker threads to run in parallel.')
#线程数量。如果数据文件很多且CPU性能足够的话，建议设置一个相对大的值
tf.app.flags.DEFINE_string('log', 'INFO',
                           'The threshold for what messages will be logged '
                           'DEBUG, INFO, WARN, ERROR, or FATAL.')
#显示消息的类型
```

这是Tensorflow中用于从命令行传递参数的变量，基于argparse实现。如果在运行时不输入参数，则会按程序中默认填写的参数运行。
因此可以看到，这一个程序共有5个参数，而上面两种命令行方法都没有写出所有的变量，但上述两种方法都能运行，因为没有默认值的变量只有输入路径和输出路径两个。
通过
```
python convert_dir_to_note_sequences.py.py –h 
```

可以显示注释信息和参数及其详情。
因此，我们在自定义参数时，既可以在命令行运行时输入：
```
python convert_dir_to_note_sequences.py --input_dir=E:\Magenta\Dataset\raw\bach --output_file=E:\Magenta\Dataset\pre\bach.tfrecord --recursive=True --num_threads=4
```
同样，我们也可以把前面这几行当做超参数变量声明，直接在程序里改（第二个参数），然后运行。
 


## 魔改-2.0

###### Magenta version:0.3.6

接下来介绍这一步的详细原理以及文件储存的数据类型。

源代码地址：
https://github.com/tensorflow/magenta/blob/master/magenta/scripts/convert_dir_to_note_sequences.py

在本程序中，大致的运行步骤为：
1. 先检测输入路径（以及子路径）中所有符合要求的文件，生成文件路径列表。
2. 再根据列表多线程的处理数据。
3. 最后再存成.tfrecord文件。

第一步对应queue_conversions(root_dir, sub_dir, pool, recursive=False)函数，在此不多展开。

第二步对应convert_midi(root_dir, sub_dir, full_file_path)、
convert_musicxml(root_dir, sub_dir, full_file_path)两个函数。顾名思义就是针对midi和xml文件的处理函数（一开始说的ABC数据处理函数未知）。它们的参数以及返回值可以在函数注释中找到详细的介绍。简单来说就是输入文件路径、文件所在文件夹路径、上一级路径，输出NoteSequence proto，一个在Magenta项目中用来表示音符序列的数据类型。

第三步则对应convert_directory(root_dir, output_file, num_threads,recursive=False)，是总的函数。

首先我们可以把这个文件导入：

In [1]:
import tensorflow as tf
import magenta as mgt
import magenta.scripts.convert_dir_to_note_sequences as cvrt

导入之后我们也可以用查看子类的方式查看它的FLAGS参数：

In [2]:
print(cvrt.FLAGS)


magenta.scripts.convert_dir_to_note_sequences:
  --input_dir: Directory containing files to convert.
  --log: The threshold for what messages will be logged DEBUG, INFO, WARN,
    ERROR, or FATAL.
    (default: 'INFO')
  --num_threads: Number of worker threads to run in parallel.
    (default: '1')
    (an integer)
  --output_file: Path to output TFRecord file. Will be overwritten if it already
    exists.
  --[no]recursive: Whether or not to recurse into subdirectories.
    (default: 'false')

absl.flags:
  --flagfile: Insert flag definitions from the given file into the command line.
    (default: '')
  --undefok: comma-separated list of flag names that it is okay to specify on
    the command line even if the program does not define a flag with that name.
    IMPORTANT: flags in this list that have arguments MUST use the --flag=value
    format.
    (default: '')


In [3]:
#加这行是因为jupyter notebook对tf.app.flags.FLAGS有bug
#见https://github.com/tensorflow/tensorflow/issues/17702
tf.app.flags.DEFINE_string('f', '', 'kernel')

因此我们也可以用修改FLAGS子类参数的方法运行本程序：

我这里以及下面的代码中都采用了绝对路径，所以在自己运行时请修改路径。文件会附在github中。

In [4]:
cvrt.FLAGS.input_dir=r'E:\Magenta\Dataset\xml_raw'
cvrt.FLAGS.output_file=r'E:\Magenta\Dataset\pre\bach.tfrecord'
cvrt.FLAGS.num_threads=4
cvrt.FLAGS.recursive=True
cvrt.FLAGS.log='INFO'

In [5]:
tf.app.run(cvrt.main)

INFO:tensorflow:Converting files in 'E:\Magenta\Dataset\xml_raw\'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converted MusicXML file E:\Magenta\Dataset\xml_raw\bwv1.6.mxl.
INFO:tensorflow:Converted MusicXML file E:\Magenta\Dataset\xml_raw\bwv2.6.mxl.


SystemExit: 

D:\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


如上所说，这个文件包含convert_midi(root_dir, sub_dir, full_file_path)、convert_musicxml(root_dir, sub_dir, full_file_path)两个函数。

下面我们分别来运行一下转换函数并看一下它们返回的结果。

In [6]:
full_file_path_xml=r'E:\Magenta\Dataset\xml_raw\bwv1.6.mxl'
root_dir_xml=r'E:\Magenta\Dataset\xml_raw'
sub_dir_xml=r'E:\Magenta\Dataset\xml_raw'
sequence_xml=cvrt.convert_musicxml(root_dir_xml, sub_dir_xml, full_file_path_xml)

INFO:tensorflow:Converted MusicXML file E:\Magenta\Dataset\xml_raw\bwv1.6.mxl.


我们可以看到sequence_xml是一个基于Google protobuf的数据类型。

In [7]:
print(type(sequence_xml))

<class 'magenta.protobuf.music_pb2.NoteSequence'>


In [8]:
#print(sequence_xml)

从上面我们可以看到这里面包含了路径、id、以及xml中的内容。大部分应该是直接从xml中直接转换而来，但将它们分成类结构化储存了。

于是，我们也可以直接访问它的子类

In [9]:
print(sequence_xml.id)
print(sequence_xml.filename)

/id/musicxml/xml_raw/efab6353dd5ebf096204bc6316d1d2f003b156c7
E:\Magenta\Dataset\xml_raw\bwv1.6.mxl


In [10]:
print(type(sequence_xml.notes))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>


sequence_xml的note类里面就是最主要的内容了，主要记录了所有的音符。
音符类当然也支持索引，我们可以看到每个音符由音高、音色、起始时间、终止时间等组成。

In [11]:
print(sequence_xml.notes[0])

pitch: 65
velocity: 64
end_time: 0.5
numerator: 1
denominator: 4
instrument: 7
program: 1
voice: 1



我用MuseScore2将XML导出为midi后进行下面的测试：

In [12]:
full_file_path_midi=r'E:\Magenta\Midi\bwv1.6.mid'
root_dir_midi=r'E:\Magenta\Midi'
sub_dir_midi=r'E:\Magenta\Midi'
sequence_midi=cvrt.convert_midi(root_dir_midi, sub_dir_midi, full_file_path_midi)

INFO:tensorflow:Converted MIDI file E:\Magenta\Midi\bwv1.6.mid.


D:\Anaconda3\envs\tensorflow\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


我们看到，MIDI形式的储存格式和XML大同小异，但是起始和终止的时间看起来很乱。

In [13]:
#print(sequence_midi)

In [14]:
print(type(sequence_midi.notes))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>


In [15]:
print(sequence_midi.notes[0])
print(sequence_midi.notes[1])
print(sequence_midi.notes[288])

pitch: 65
velocity: 80
end_time: 0.7878292625

pitch: 67
velocity: 80
start_time: 0.789474
end_time: 1.1825662625

pitch: 67
velocity: 80
start_time: 51.31581
end_time: 52.1036392625
instrument: 2



我们可以看到，MIDI的Notes里面的内容就相对简单了。而且似乎格式也不太整齐。这有可能是由于我转换的方式不够好。

In [16]:
full_file_path_midi=r'E:\Magenta\Midi\Bwv0525 Sonate en trio n1.mid'
root_dir_midi=r'E:\Magenta\Midi'
sub_dir_midi=r'E:\Magenta\Midi'
sequence_midi=cvrt.convert_midi(root_dir_midi, sub_dir_midi, full_file_path_midi)

INFO:tensorflow:Converted MIDI file E:\Magenta\Midi\Bwv0525 Sonate en trio n1.mid.


In [17]:
#print(sequence_midi)

In [18]:
print(sequence_midi.notes[0])
print(sequence_midi.notes[1])
print(sequence_midi.notes[2880])

pitch: 70
velocity: 92
start_time: 6.4
end_time: 6.800000000000001

pitch: 74
velocity: 92
start_time: 6.800000000000001
end_time: 7.2

pitch: 69
velocity: 97
start_time: 139.8
end_time: 140.0
instrument: 1
program: 19



换了一个MIDI数据集中的MIDI文件，似乎还是这样。具体原因我再寻找一下，在下一节时分析。

## 总结
我们了解了Magenta项目原始数据整合的过程，并了解了读取MIDI和XML的函数。

如果你想进行自己的项目的话，直接用Magenta的数据处理函数也是个不错的选择。

同时，也有其他的数据读取与处理方式，例如Magenta.piplines类，pretty_midi库，都是不错的选择。但在这里就不详细展开了，也许在之后的教程里会提到。

对于读取到的MIDI数据以及XML数据的解释，我会在下一节中说明。

最后，对于生成的tfrecord，Magenta将数据转换成了二进制字符储存，读取稍微有些复杂，我们有机会再详细说明。